# Backups
### Boilerplate - Get session Info
- Using API key make a request get ascociated `session_info` 
- `session_info` contains current user and current customer uuid.

In [136]:
import http.client
import json
import os
from pprint import pprint

platform_api_key = os.getenv('YB_API_KEY')
platform_address = os.getenv('API_BASE_URL', "portal.dev.yugabyte.com")

conn = http.client.HTTPConnection(f"{platform_address}")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{platform_api_key}"
}

conn.request("GET", "/api/v1/session_info", headers=headers)

res = conn.getresponse()
data = res.read()
session_info = json.loads(data)
pprint(session_info)

{'apiToken': '753aef54-8fbb-442d-a063-e41b3a62b7e2',
 'customerUUID': '11d78d93-1381-4d1d-8393-ba76f47ba7a6',
 'userUUID': '52bce00c-dd1b-dcd7-0821-b8070a7173c5'}


#### Storage Configuration Parameter
- Now pick a storage config you want to use to store the backups.
See `./list-customer-config.ipynb` for how to list all configs

In [137]:
storage_config_uuid = os.getenv('STORAGE_CONF_UUID',
                                "f5cfe9fb-cd4f-46e7-8ed8-af285b730cc0")

#### Universe UUID Parameter
uuid of universe to be backed up. See `list-universes.ipynb` for how to

In [138]:
universe_uuid = os.getenv('UNIVERSE_UUID',
                          "303cf5ac-a6f5-492b-b68c-7d4d9633c4cc")

### Create one-time (i.e. unscheduled) Whole Universe Backup
- delete (garbage collect) backup after 10 days (864000000 ms)

In [128]:

payload = ('{\n'
           '  "actionType": "CREATE",\n'
           '  "keyspace": "postgres",\n'
           f'  "storageConfigUUID": "{storage_config_uuid}",\n'
           '  "timeBeforeDelete": "864000000",\n'
           '  "sse": "false",\n'
           '  "transactionalBackup": "false",\n'
           '  "parallelism": "8",\n'
           '  "backupType": "PGSQL_TABLE_TYPE"\n'
           '}')

url = f"/api/v1/customers/{session_info.get('customerUUID')}" \
      f"/universes/{universe_uuid}/multi_table_backup"
conn.request("PUT", url, payload, headers)

res = conn.getresponse()
yb_task = json.load(res)
pprint(yb_task)

{'taskUUID': '4c66a198-bcc7-4b24-973d-1289654a1b92'}


### List tables

#### table_name
- We assume that you have created table with name "cassandrakeyvalue" using [yb-sample-app](https://github.com/yugabyte/yb-sample-apps)

In [155]:
table_name = "cassandrakeyvalue"

#### table_name
- We will list tables and get the uuid of table with name "cassandrakeyvalue"

In [156]:
url = f"/api/v1/customers/{session_info.get('customerUUID')}" \
      f"/universes/{universe_uuid}/tables"
conn.request("GET", url, headers=headers)
res = conn.getresponse()
data = res.read()
table_list = json.loads(data)
for table in table_list:
    if table['tableName'] == table_name:
        break
pprint(table)

{'isIndexTable': False,
 'keySpace': 'ybdemo_keyspace',
 'relationType': 'USER_TABLE_RELATION',
 'sizeBytes': 12320595.0,
 'tableName': 'cassandrakeyvalue',
 'tableType': 'YQL_TABLE_TYPE',
 'tableUUID': '229c8815-2273-4bc5-9396-eaf02eae4a6d'}


### Create one-time (i.e. unscheduled) single table backup

In [157]:
payload = ('{\n'
           '  "actionType": "CREATE",\n'
           f'  "keyspace": "{table.get("keySpace")}",\n'
           f'  "tableName": "{table_name}",\n'
           f'  "storageConfigUUID": "{storage_config_uuid}",\n'
           '  "timeBeforeDelete": "864000000"\n'
           '}')
print(payload)
url = f"/api/v1/customers/{session_info.get('customerUUID')}" \
      f"/universes/{universe_uuid}/tables/{table['tableUUID']}/create_backup"
print(url)
conn.request("PUT", url, payload, headers)

res = conn.getresponse()
yb_task = json.load(res)
pprint(yb_task)

{
  "actionType": "CREATE",
  "keyspace": "ybdemo_keyspace",
  "tableName": "cassandrakeyvalue",
  "storageConfigUUID": "f5cfe9fb-cd4f-46e7-8ed8-af285b730cc0",
  "timeBeforeDelete": "864000000"
}
/api/v1/customers/11d78d93-1381-4d1d-8393-ba76f47ba7a6/universes/303cf5ac-a6f5-492b-b68c-7d4d9633c4cc/tables/229c8815-2273-4bc5-9396-eaf02eae4a6d/create_backup
{'resourceUUID': 'b72c7308-8654-46ac-9f68-f95ead72f48b',
 'taskUUID': 'da65ebca-2010-41ee-a432-0320c46aa931'}


### Stop an ongoing backup

In [158]:
url = f"/api/v1/customers/{session_info.get('customerUUID')}" \
      f"/backups/{yb_task.get('resourceUUID')}/stop"
print(url)
conn.request("POST", url, "{}", headers=headers)
res = conn.getresponse()
print(json.loads(res.read()))

/api/v1/customers/11d78d93-1381-4d1d-8393-ba76f47ba7a6/backups/b72c7308-8654-46ac-9f68-f95ead72f48b/stop
{'success': False, 'error': 'The process you want to stop is not in progress.'}


### Delete a the completed backup

In [160]:
url = f"/api/v1/customers/{session_info.get('customerUUID')}" \
      f"/backups"
payload = ('{\n'
           f'  "backupUUID": ["{yb_task.get("resourceUUID")}"]\n'
           '}')
print(payload)
conn.request("DELETE", url, payload, headers=headers)
res = conn.getresponse()
print(res.read())

{
  "backupUUID": ["b72c7308-8654-46ac-9f68-f95ead72f48b"]
}
b'{"taskUUID":["81ce8e04-35ad-4e38-b5f3-d63b479ce061"],"ybpTaskList":[{"taskUUID":"81ce8e04-35ad-4e38-b5f3-d63b479ce061","resourceUUID":"b72c7308-8654-46ac-9f68-f95ead72f48b"}]}'


### Create Scheduled Whole Universe Backup
- *schedulingFrequency* (i.e. take backup every) 1 day (86400000 ms) 
- delete (garbage collect) backup after 10 days (864000000 ms)

In [165]:

payload = ('{\n'
           '  "actionType": "CREATE",\n'
           '  "keyspace": "postgres",\n'
           f'  "storageConfigUUID": "{storage_config_uuid}",\n'
           '  "timeBeforeDelete": "864000000",\n'
           '  "sse": "false",\n'
           '  "transactionalBackup": "false",\n'
           '  "parallelism": "8",\n'
           '  "schedulingFrequency": "86400000",\n'
           '  "backupType": "PGSQL_TABLE_TYPE"\n'
           '}')

url = f"/api/v1/customers/{session_info.get('customerUUID')}" \
      f"/universes/{universe_uuid}/multi_table_backup"
conn.request("PUT", url, payload, headers)

res = conn.getresponse()
schedule_info = json.load(res)
pprint(schedule_info)

{'customerUUID': '11d78d93-1381-4d1d-8393-ba76f47ba7a6',
 'failureCount': 0,
 'frequency': 86400000,
 'scheduleUUID': '8358152a-bd05-484e-8138-af4ea4b3595b',
 'status': 'Active',
 'taskParams': {'actionType': 'CREATE',
                'backupType': 'PGSQL_TABLE_TYPE',
                'communicationPorts': {'masterHttpPort': 7000,
                                       'masterRpcPort': 7100,
                                       'nodeExporterPort': 9300,
                                       'redisServerHttpPort': 11000,
                                       'redisServerRpcPort': 6379,
                                       'tserverHttpPort': 9000,
                                       'tserverRpcPort': 9100,
                                       'yqlServerHttpPort': 12000,
                                       'yqlServerRpcPort': 9042,
                                       'ysqlServerHttpPort': 13000,
                                       'ysqlServerRpcPort': 5433},
           

### Delete backup Schedule

In [166]:
url = f"/api/v1/customers/{session_info.get('customerUUID')}" \
      f"/schedules/{schedule_info.get('scheduleUUID')}"
conn.request("DELETE", url, headers=headers)
res = conn.getresponse()
print(res.read())

b'{"success":true}'
